In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!pip install bayesian-optimization

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, KFold # KFold needs to be imported
from scipy.stats import pearsonr
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Layer, Multiply, Add, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2


import tensorflow as tf
tf.config.run_functions_eagerly(True)

from bayes_opt import BayesianOptimization

# Data
loadings_df = pd.read_csv('') #the path for the loadings
micedata_df = pd.read_csv('') ##the path for the genomic data

# processing
micedata_features = micedata_df.iloc[:, 2:].values
loadings_real = loadings_df.applymap(lambda x: np.real(complex(x)) if isinstance(x, str) else np.real(x)).values
#loadings_real = loadings_df.map(lambda x: np.real(complex(x)) if isinstance(x, str) else np.real(x)).values

print(f"Shape of micedata_features: {micedata_features.shape}")
print(f"Shape of loadings_real: {loadings_real.shape}")


# Feature Reduction for loadings ONLY
loadings_reduced = loadings_real

# Combine features
combined_features = np.hstack((micedata_features, loadings_reduced))

print(f"Shape of combined_features after concatenation: {combined_features.shape}")

scaler = StandardScaler()
normalized_features = scaler.fit_transform(combined_features)
target = micedata_df.iloc[:, :2].values

# Define the layers
class GatedResidualUnit(Layer):
    def __init__(self, units, **kwargs):
        super(GatedResidualUnit, self).__init__(**kwargs)
        self.units = units
        self.dense = Dense(units, activation='elu', kernel_regularizer=l2(0.01))
        self.gate = Dense(units, activation='sigmoid', kernel_regularizer=l2(0.01))
        # This layer will project the residual connection if its shape doesn't match 'units'
        self.residual_projection = None # Initialize as None, build later if needed

    def build(self, input_shape):
        if input_shape[-1] != self.units:
            self.residual_projection = Dense(self.units, activation='linear', kernel_regularizer=l2(0.01))
        else:
            self.residual_projection = tf.identity # Use identity if shapes match, no projection needed
        super(GatedResidualUnit, self).build(input_shape)

    def call(self, inputs):
        residual = inputs
        x = self.dense(inputs)
        gate = self.gate(inputs)
        gated_output = Multiply()([x, gate])

        
        if self.residual_projection is not tf.identity: 
            projected_residual = self.residual_projection(residual)
        else:
            projected_residual = residual 

        return Add()([gated_output, projected_residual])

class VariableSelectionNetwork(Layer):
    def __init__(self, units, **kwargs):
        super(VariableSelectionNetwork, self).__init__(**kwargs)
        self.units = units
        self.dense = Dense(units, activation='sigmoid', kernel_regularizer=l2(0.01))

    def call(self, inputs):
        if inputs.shape[1] < self.units:
            inputs_for_selection = inputs
        else:
            inputs_for_selection = inputs[:, :self.units]

        dense_output = self.dense(inputs_for_selection)
        hard_sigmoid_output = tf.maximum(0.0, tf.minimum(1.0, 0.2 * dense_output + 0.5))
        return Multiply()([inputs_for_selection, hard_sigmoid_output])

# creat the grvsnn model
def create_grn_vsn_model(input_shape, dropout_rate=0.3, n_hidden_units = 128):
    inputs = Input(shape=input_shape)

    #step 1: GR block
    grn_output = GatedResidualUnit(units = n_hidden_units)(inputs)
    grn_output = Dropout(dropout_rate)(grn_output)

    #step 2: VS block
    selected_features = VariableSelectionNetwork(units = n_hidden_units)(grn_output)
    outputs = Dense(2)(selected_features)
    model = Model(inputs, outputs)
    return model

# BO
def train_model(lr, dropout_rate):
    lr = max(0.00001, min(lr, 0.001))
    dropout_rate = max(0.1, min(dropout_rate, 0.5))

    kf = KFold(n_splits=5, shuffle=True, random_state=42) #n_splits can be 5 or 10. in our case, we use 10-fold
    mse_scores = []

    for train_index, val_index in kf.split(normalized_features):
        X_train, X_val = normalized_features[train_index], normalized_features[val_index]
        y_train, y_val = target[train_index], target[val_index]

        model = create_grn_vsn_model(input_shape=(normalized_features.shape[1],), dropout_rate=dropout_rate)

        optimizer = Adam(learning_rate=lr)
        model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mae'])

        early_stopping = EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True)
        model.fit(X_train, y_train, epochs=100, batch_size=128, validation_data=(X_val, y_val), callbacks=[early_stopping], verbose=0)

        loss, _ = model.evaluate(X_val, y_val, verbose=0)
        mse_scores.append(loss)

    return -np.mean(mse_scores)

optimizer = BayesianOptimization(f=train_model, pbounds={'lr': (0.00001, 0.001), 'dropout_rate': (0.1, 0.5)}, verbose=2)
optimizer.maximize(init_points=5, n_iter=10)

# 8. Training and test
best_params = optimizer.max['params']
kf = KFold(n_splits=5, shuffle=True, random_state=42)

final_mse_scores = []
for train_index, val_index in kf.split(normalized_features):
    X_train, X_val = normalized_features[train_index], normalized_features[val_index]
    y_train, y_val = target[train_index], target[val_index]

    final_model = create_grn_vsn_model(input_shape=(normalized_features.shape[1],), dropout_rate=best_params['dropout_rate'])
    final_model.compile(optimizer=Adam(learning_rate=best_params['lr']), loss='mean_squared_error',
                         metrics=[tf.keras.metrics.MeanSquaredError(name='mse_col1'), tf.keras.metrics.MeanSquaredError(name='mse_col2')])

    early_stopping = EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True)
    final_model.fit(X_train, y_train, epochs=100, batch_size=128, validation_data=(X_val, y_val), callbacks=[early_stopping])

    loss, mse_col1, mse_col2 = final_model.evaluate(X_val, y_val)
    final_mse_scores.append(loss)
    print(f"Fold test MSE: {loss}\nTrait 1 test MSE: {mse_col1}\nTrait 2 test MSE: {mse_col2}")

print(f"Average Test MSE across 5 folds: {np.mean(final_mse_scores)}")